In [9]:
import numpy as np
from openqaoa.problems import BinPacking, MaximumCut, TSP, ShortestPath
import json
import os
import networkx as nx

In [10]:
def generate_results_filename(problem_type, num_qubits):
    return f"batch_QUBO_data_{str(problem_type).replace(' ', '_')}_{num_qubits}q_.json"

In [11]:
desiredProblemType = 'MaxCut' #options: 'BinPacking', 'MaxCut', 'TSP', 'ShortestPath'
#make sure you typed in that mf desiredProblemType correctly
batchSize = 5
RESULTS_FOLDER = "QUBO_batches"

In [12]:
all_qubos_list = []
for instance in range(batchSize):
    if desiredProblemType == 'BinPacking': #doesnt seem possible to get a non-trivial example of this problem in less than 12 qubits
        n_items = 2 # number of items
        n_bins = 2 # maximum number of bins the solution will be explored on
        min_weight = 1 # minimum weight of the items
        max_weight = 3 # maximum weight of the items
        weight_capacity = 5 # weight capacity of the bins
        weights = np.random.default_rng(seed=1234).integers(low=min_weight, high=max_weight, size=n_items) # random instance of the problem

        bpp = BinPacking(weights, weight_capacity, n_bins=n_bins, simplifications=False) #it is here that you would specifiy the penalty for the bin packing problem
        quboProb = bpp.qubo

    elif desiredProblemType == 'MaxCut':
        G = nx.generators.fast_gnp_random_graph(n=20, p=0.6) #you can fix the seed for the graphs here
        maxcut_prob = MaximumCut(G)
        quboProb = maxcut_prob.qubo
        

    elif desiredProblemType == 'TSP':
        n_cities = 4  # Number of cities for TSP
        connection_probability = 1  # Probability for edge creation in gnp_random_graph
        # Ensure the graph is connected
        G = nx.generators.fast_gnp_random_graph(n=n_cities, p=connection_probability) 

        min_edge_weight = 1
        max_edge_weight = 10 
        rng_weights = np.random.default_rng() # Use a different seed or manage seeds as needed
        for (u, v) in G.edges():
            weight = int(rng_weights.integers(low=min_edge_weight, high=max_edge_weight, endpoint=True)) # endpoint=True includes high value
            G.edges[u, v]['weight'] = weight
        # A is the penalty for not visiting a city or visiting it multiple times.
        # B is the penalty for not having a valid tour structure (e.g. sub-tours or incorrect number of cities at a position)
        B_val = 1
        A_val = 9
        tsp_prob = TSP(G=G, A=A_val, B=B_val) # Using your specified A and B penalties
        quboProb = tsp_prob.qubo
    

    elif desiredProblemType == 'ShortestPath': #not sure that the exact solver is working right for this one
        G = nx.generators.fast_gnp_random_graph(n=6, p=0.6)
        nx.set_edge_attributes(G, values = 1, name = 'weight')
        nx.set_node_attributes(G, values = 1, name = 'weight')
        sp_prob = ShortestPath(G, source=0, dest=5)
        quboProb = sp_prob.qubo
    
    print(f"Generated QUBO for {desiredProblemType} instance {instance + 1} of {batchSize}", end="\r")
    qubo_dict = quboProb.asdict()
    current_qubo_data = {
        "instance_id": instance + 1, # Add an identifier for each QUBO instance
        "terms": qubo_dict["terms"],
        "weights": qubo_dict["weights"],
        "constant": qubo_dict.get("constant", 0.0),
        "problem_type": qubo_dict.get("problem_instance", {}).get("problem_type", "unknown"),
        # Optional: include more problem-specific details if needed for later analysis, can add the number of qubits/problem size
    }
 
    all_qubos_list.append(current_qubo_data) 

all_indices = []
terms = qubo_dict["terms"]
for term_group in terms:
    for idx in term_group:
        all_indices.append(idx)
num_qubits = max(all_indices) + 1

file_name = generate_results_filename(desiredProblemType, num_qubits)
results_filename_with_path = os.path.join(RESULTS_FOLDER, file_name)

with open(results_filename_with_path, "w") as f:
    f.write("[\n") # Start of the JSON array
    for i, single_qubo_data in enumerate(all_qubos_list):
        qubo_json_string = json.dumps(single_qubo_data)
        f.write("  " + qubo_json_string)
        
        if i < len(all_qubos_list) - 1:
            f.write(",\n")
        else:
            f.write("\n")
            
    f.write("]\n") 

print(f"\nBatch of {len(all_qubos_list)} QUBOs saved to {results_filename_with_path}")
    

Generated QUBO for MaxCut instance 5 of 5
Batch of 5 QUBOs saved to QUBO_batches/batch_QUBO_data_MaxCut_20q_.json
